In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import networkx as nx

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
# nltk.download('punkt')

from summarizer import Summarizer

[nltk_data] Downloading package punkt to /home/isha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Method 1
---

In [ ]:
text = open('file1.txt','r').read()
model = Summarizer()
result = model(text, min_length=20)
summary = "".join(result)
print(summary)

## Method 2
---

In [11]:
df = pd.read_csv('file1.txt', header=None, delimiter="\t")
df[0]

0                         Item 1. Business Description
1    Berkshire Hathaway Inc. (“Berkshire,” “Company...
2    Berkshire’s operating businesses are managed o...
3    Berkshire’s senior management is also responsi...
4    Berkshire and its consolidated subsidiaries em...
5                 Insurance and Reinsurance Businesses
6    Berkshire’s insurance and reinsurance business...
7    In direct or primary insurance activities, the...
8    Reinsurance contracts are normally classified ...
9    Insurance and reinsurance are generally subjec...
Name: 0, dtype: object

In [14]:
allSentences = []
for sentence in df[0]:
    allSentences.append(sent_tokenize(sentence))
allSentences = [y for x in allSentences for y in x]
allSentences

['Item 1. Business Description',
 'Berkshire Hathaway Inc. (“Berkshire,” “Company” or “Registrant”) is a holding company owning subsidiaries engaged in a large number of diverse business activities.',
 'The most important of these are insurance businesses conducted on both a primary basis and a reinsurance basis, a freight rail transportation business and a group of utility and energy generation and distribution businesses.',
 'Berkshire also owns and operates numerous other businesses engaged in a variety of activities, as identified herein.',
 'Berkshire is domiciled in the state of Delaware, and its corporate headquarters is in Omaha, Nebraska.',
 'Berkshire’s operating businesses are managed on an unusually decentralized basis.',
 'There are few centralized or integrated business functions.',
 'Berkshire’s corporate senior management team participates in and is ultimately responsible for significant capital allocation decisions, investment activities and the selection of the Chief 

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-03-08 10:50:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-08 10:50:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-08 10:50:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [15]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [17]:
clean_sentences = pd.Series(allSentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]

In [18]:
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [19]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [20]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [21]:
sim_mat = np.zeros([len(allSentences), len(allSentences)])

In [23]:
for i in range(len(allSentences)):
    for j in range(len(allSentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [24]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [26]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(allSentences)), reverse=True)

In [28]:
for i in range(5):
    print(ranked_sentences[i][1])

Berkshire’s insurance and reinsurance business activities are conducted through numerous domestic and foreign-based insurance entities.
In direct or primary insurance activities, the insurer assumes the risk of loss from persons or organizations that are directly subject to the risks.
The most important of these are insurance businesses conducted on both a primary basis and a reinsurance basis, a freight rail transportation business and a group of utility and energy generation and distribution businesses.
Consistent with Berkshire’s decentralized management philosophy, Berkshire’s operating businesses establish specific policies and practices for their businesses concerning the attraction and retention of personnel within the organizations.
Insurance and reinsurance are generally subject to regulatory oversight throughout the world.
